This notebook contains the code for the meta-analysis of healthy lung data for ACE2, TMPRSS2, and CTSL. It contains the simple model without interaction terms that was run on the cell-level data (not pseudo-bulk, no holdout analysis). This script contains the code that was run on the full data and does not test for smoking associations.

In [1]:
import scanpy as sc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
from matplotlib import colors
from matplotlib import patches
import seaborn as sns
import batchglm
import diffxpy.api as de
import patsy as pat
from statsmodels.stats.multitest import multipletests
import logging, warnings
import statsmodels.api as sm

/home/icb/malte.luecken/anaconda3/envs/cov19_py/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()
de.__version__

logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("batchglm").setLevel(logging.INFO)
logging.getLogger("diffxpy").setLevel(logging.INFO)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 35)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")

scanpy==1.4.5.1 anndata==0.7.1 umap==0.3.10 numpy==1.18.1 scipy==1.4.1 pandas==1.0.1 scikit-learn==0.23.1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


'v0.7.3'

In [3]:
#User inputs
folder = '/storage/groups/ml01/workspace/malte.luecken/2020_cov19_study'

adata_diffxpy = '/storage/groups/ml01/workspace/malte.luecken/2020_cov19_study/COVID19_lung_atlas_revision_v3.h5ad'

output_folder = 'diffxpy_out/'

de_output_base = 'COVID19_lung_atlas_revision_v3_lung_cov19_poissonglm_nUMIoffset_noInts'

# Read the data

In [4]:
adata = sc.read(adata_diffxpy)

In [5]:
adata

AnnData object with n_obs × n_vars = 1320896 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new'

In [6]:
adata.obs.age = adata.obs.age.astype(float)

In [7]:
adata.obs.dtypes

age                         float64
anatomical_region          category
donor                      category
last_author/PI             category
lung_vs_nasal              category
notes                      category
original_celltype_ann      category
sample                     category
sex                        category
smoking                    category
total_counts                float64
smoked_boolean             category
last_author_sample_name    category
ann_level_1                category
ann_level_2                category
ann_level_3                category
ann_level_4                category
ann_level_5                category
ann_highest_res               int64
ann_new                        bool
dtype: object

In [8]:
adata.obs['dataset'] = adata.obs['last_author/PI']

In [9]:
adata.obs.dataset.value_counts()

Regev/Rajagopal            322998
Meyer_b                    117535
Kaminski                    95303
Spence                      78401
Barbry/Leroy                76981
Krasnow/Quake               60993
Meyer                       57020
Rawlins                     53409
Regev                       43527
Misharin/Budinger           41266
Eils/Conrad/Kreuter         39778
Seibold                     36248
Whitsett/Xu_10X             34185
Koenigshoff                 33119
Misharin                    28329
Xavier/Regev                25552
Spira/Campbell              24455
Lafyatis/Rojas              24220
Kropski/Banovich_vand       23285
Schultze                    22641
Schiller                    20776
Nawijn                      18197
Teichmann                   12971
Kropski/Banovich_dnar        8359
Shalek                       7603
Linnarsson                   4640
Whitsett/Xu_dropSeq          3267
Mazzilli/Campbell/Beane      2207
Schultze/Falk                1965
Beane         

# Filter the data

Keep only datsets with:
- more than 1 donor
- non-fetal
- lung

In [10]:
# Remove fetal datasets
dats_to_remove = set(['Rawlins', 'Spence', 'Linnarsson'])

In [11]:
dat = adata.obs.groupby(['donor']).agg({'sex':'first', 'age':'first', 'dataset':'first'})

# Single donor filter
don_tab = dat['dataset'].value_counts()
dats_to_remove.update(set(don_tab.index[don_tab == 1]))

In [12]:
dats_to_remove = list(dats_to_remove)
dats_to_remove

['Misharin', 'Schultze/Falk', 'Spence', 'Rawlins', 'Linnarsson']

In [13]:
adata = adata[~adata.obs.dataset.isin(dats_to_remove)].copy()

In [14]:
adata.obs.lung_vs_nasal.value_counts()

lung     1096604
nasal      57548
Name: lung_vs_nasal, dtype: int64

In [15]:
# Filter for only lung data
adata = adata[adata.obs.lung_vs_nasal.isin(['lung']),].copy()

In [16]:
adata

AnnData object with n_obs × n_vars = 1096604 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset'

In [17]:
adata.obs.dataset.value_counts()
adata.obs['sample'].nunique()
adata.obs['donor'].nunique()

Regev/Rajagopal            322998
Meyer_b                    117535
Kaminski                    95303
Krasnow/Quake               60993
Barbry/Leroy                58790
Meyer                       57020
Regev                       43527
Misharin/Budinger           41266
Eils/Conrad/Kreuter         39778
Seibold                     36248
Whitsett/Xu_10X             34185
Koenigshoff                 33119
Xavier/Regev                25552
Lafyatis/Rojas              24220
Kropski/Banovich_vand       23285
Schultze                    22641
Schiller                    20776
Teichmann                   12971
Nawijn                      11110
Kropski/Banovich_dnar        8359
Whitsett/Xu_dropSeq          3267
Mazzilli/Campbell/Beane      1995
Beane                         886
Beane/Campbell                780
Name: dataset, dtype: int64

309

185

# Check the data

In [18]:
np.mean(adata.X.astype(int) != adata.X)

0.0

In [19]:
# Check if any non-integer data in a particular dataset
for dat in adata.obs.dataset.unique():
    val = np.mean(adata[adata.obs.dataset.isin([dat]),:].X.astype(int) != adata[adata.obs.dataset.isin([dat]),:].X)
    if val != 0:
        print(f'dataset= {dat}; value= {val}')
        adata[adata.obs.dataset.isin([dat]),:].X[:20,:20].A

All counts are integers

In [20]:
adata.obs.age.value_counts()
adata.obs.sex.value_counts()

57.00    81833
66.00    69371
42.00    66596
59.00    59955
18.00    59849
46.00    56291
64.00    46130
35.00    45869
0.25     29908
67.50    28201
55.50    27176
20.00    25362
51.00    24766
72.50    24366
30.00    24095
58.00    23723
29.00    19248
0.00     19065
32.00    19009
3.00     18387
42.50    16906
45.50    16351
56.00    16011
23.00    15181
41.00    14943
49.00    14225
57.50    13840
55.00    13733
68.00    11852
65.00    11404
75.00    11243
45.00    10939
47.00    10278
52.50    10133
21.00     9430
67.00     9278
27.00     8693
22.00     7980
63.00     7600
31.00     7085
33.00     6906
44.00     6681
62.00     6228
38.00     5629
73.00     5251
24.00     4433
60.00     4231
62.50     4073
26.00     4066
61.00     3922
50.00     3831
32.50     3772
52.00     3765
80.00     3261
10.00     2694
40.00     2647
17.00     2552
79.00     2550
54.00     2507
76.00     2447
36.00     2218
53.00     1592
37.00     1240
48.00     1153
81.00     1145
25.00      823
70.00     

female    583926
male      512678
Name: sex, dtype: int64

# Fit models and perform DE

In [21]:
cluster_key = 'ann_level_2'
clust_tbl = adata.obs[cluster_key].value_counts()
clusters = clust_tbl.index[clust_tbl > 1000]
ct_to_rm = clusters[[ct.startswith('1') for ct in clusters]]
clusters = clusters.drop(ct_to_rm.tolist()).tolist()
clusters

['Myeloid',
 'Airway epithelium',
 'Alveolar epithelium',
 'Lymphoid',
 'Fibroblast lineage',
 'Blood vessels',
 'Submucosal Gland',
 'Smooth Muscle',
 'Lymphatics',
 'Mesothelium',
 'Endothelial-like',
 'Granulocytes']

Calculate DE genes per cluster.

In [22]:
adata

AnnData object with n_obs × n_vars = 1096604 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset'

In [23]:
adata.obs['total_counts_scaled'] = adata.obs['total_counts']/adata.obs['total_counts'].mean()

In [24]:
formula = "1 + sex + age + dataset"
tested_coef = ["sex[T.male]", "age"]
dmat = de.utils.design_matrix(
    data=adata,
    formula="~" + formula,
    as_numeric=["age"],
    return_type="patsy"
)
dmat[1]

['Intercept',
 'sex[T.male]',
 'dataset[T.Beane]',
 'dataset[T.Beane/Campbell]',
 'dataset[T.Eils/Conrad/Kreuter]',
 'dataset[T.Kaminski]',
 'dataset[T.Koenigshoff]',
 'dataset[T.Krasnow/Quake]',
 'dataset[T.Kropski/Banovich_dnar]',
 'dataset[T.Kropski/Banovich_vand]',
 'dataset[T.Lafyatis/Rojas]',
 'dataset[T.Mazzilli/Campbell/Beane]',
 'dataset[T.Meyer]',
 'dataset[T.Meyer_b]',
 'dataset[T.Misharin/Budinger]',
 'dataset[T.Nawijn]',
 'dataset[T.Regev]',
 'dataset[T.Regev/Rajagopal]',
 'dataset[T.Schiller]',
 'dataset[T.Schultze]',
 'dataset[T.Seibold]',
 'dataset[T.Teichmann]',
 'dataset[T.Whitsett/Xu_10X]',
 'dataset[T.Whitsett/Xu_dropSeq]',
 'dataset[T.Xavier/Regev]',
 'age']

## Poisson GLM

In [25]:
# Poisson GLM loop
de_results_lvl2_glm = dict()

# Test over clusters
for clust in clusters:
    adata_tmp = adata[adata.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=10)
    if adata_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_tmp.obs.sex.value_counts())==1:
        print(f'{clust} only has 1 type of male/female sample.')
        continue
        
    print(f'Testing {adata_tmp.n_vars} genes...')
    print(f'Testing in {adata_tmp.n_obs} cells...')
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_tmp, #[idx_train],
        formula="~" + formula,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_tmp.X[:, i].todense(), #[idx_train, :], 
            exog=dmat[0], 
            offset=np.log(adata_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()


        # Test over coefs
        for coef in tested_coef:
            de_results_temp = pois_results.wald_test(
                [x for i, x in enumerate(pois_model.exog_names) if dmat[1][i] in [coef]]
            )

            # Output the results nicely
            de_results_temp = pd.DataFrame({
                "gene": gene,
                "cell_identity": clust,
                "covariate": coef,
                "coef": pois_results.params[[y == coef for y in dmat[1]]],
                "coef_sd": pois_results.bse[[y == coef for y in dmat[1]]],                 
                "pval": de_results_temp.pvalue
            }, index= [clust+"_"+gene+"_"+coef])

            de_results_list.append(de_results_temp)

    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl2_glm[clust] = de_results
    
# Join the dataframes:
full_res_lvl2_glm = pd.concat([de_results_lvl2_glm[i] for i in de_results_lvl2_glm.keys()], ignore_index=True)

In cluster Myeloid:
male      152540
female    130927
Name: sex, dtype: int64
Testing 3 genes...
Testing in 283467 cells...

In cluster Airway epithelium:
female    126940
male      100632
Name: sex, dtype: int64
Testing 3 genes...
Testing in 227572 cells...

In cluster Alveolar epithelium:
female    135518
male       86649
Name: sex, dtype: int64
Testing 3 genes...
Testing in 222167 cells...

In cluster Lymphoid:
female    85352
male      57089
Name: sex, dtype: int64
Testing 3 genes...
Testing in 142441 cells...

In cluster Fibroblast lineage:
female    32623
male      25998
Name: sex, dtype: int64
Testing 3 genes...
Testing in 58621 cells...

In cluster Blood vessels:
male      38986
female    12654
Name: sex, dtype: int64
Testing 3 genes...
Testing in 51640 cells...

In cluster Submucosal Gland:
female    19901
male      13760
Name: sex, dtype: int64
Testing 3 genes...
Testing in 33661 cells...

In cluster Smooth Muscle:
female    9589
male      6904
Name: sex, dtype: int64
Testing

## Inspect some results

In [26]:
de_results_lvl2_glm.keys()

dict_keys(['Myeloid', 'Airway epithelium', 'Alveolar epithelium', 'Lymphoid', 'Fibroblast lineage', 'Blood vessels', 'Submucosal Gland', 'Smooth Muscle', 'Lymphatics', 'Mesothelium', 'Granulocytes'])

In [27]:
full_res_lvl2_glm

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],0.534434,0.178015,2.680519e-03,4.020779e-03
1,ACE2,Myeloid,age,0.001585,0.005836,7.859520e-01,7.859520e-01
2,TMPRSS2,Myeloid,sex[T.male],0.551324,0.040177,7.472898e-43,2.241870e-42
3,TMPRSS2,Myeloid,age,-0.002224,0.001466,1.292802e-01,1.551362e-01
4,CTSL,Myeloid,sex[T.male],-0.102910,0.002132,0.000000e+00,0.000000e+00
5,CTSL,Myeloid,age,-0.000946,0.000071,3.866532e-40,7.733064e-40
6,ACE2,Airway epithelium,sex[T.male],0.063529,0.035562,7.403222e-02,8.883866e-02
7,ACE2,Airway epithelium,age,0.003375,0.001056,1.402285e-03,2.103428e-03
8,TMPRSS2,Airway epithelium,sex[T.male],0.186310,0.011434,1.076613e-59,6.459678e-59
9,TMPRSS2,Airway epithelium,age,0.000354,0.000325,2.767636e-01,2.767636e-01


In [28]:
full_res_lvl2_glm.loc[full_res_lvl2_glm['gene'] == 'ACE2',]

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],0.534434,0.178015,2.680519e-03,4.020779e-03
1,ACE2,Myeloid,age,0.001585,0.005836,7.859520e-01,7.859520e-01
6,ACE2,Airway epithelium,sex[T.male],0.063529,0.035562,7.403222e-02,8.883866e-02
7,ACE2,Airway epithelium,age,0.003375,0.001056,1.402285e-03,2.103428e-03
12,ACE2,Alveolar epithelium,sex[T.male],0.813501,0.051106,4.758037e-57,1.427411e-56
13,ACE2,Alveolar epithelium,age,0.014620,0.001491,1.063568e-22,1.063568e-22
18,ACE2,Lymphoid,sex[T.male],0.309828,0.332550,3.515054e-01,3.515054e-01
19,ACE2,Lymphoid,age,-0.028230,0.011272,1.226230e-02,1.839345e-02
24,ACE2,Fibroblast lineage,sex[T.male],0.036372,0.161040,8.213142e-01,8.213142e-01
25,ACE2,Fibroblast lineage,age,0.003974,0.005563,4.750683e-01,5.700820e-01


In [29]:
full_res_lvl2_glm.loc[(full_res_lvl2_glm['gene'] == 'ACE2') & (full_res_lvl2_glm['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],0.534434,0.178015,2.680519e-03,4.020779e-03
7,ACE2,Airway epithelium,age,0.003375,0.001056,1.402285e-03,2.103428e-03
12,ACE2,Alveolar epithelium,sex[T.male],0.813501,0.051106,4.758037e-57,1.427411e-56
13,ACE2,Alveolar epithelium,age,0.014620,0.001491,1.063568e-22,1.063568e-22
19,ACE2,Lymphoid,age,-0.028230,0.011272,1.226230e-02,1.839345e-02
30,ACE2,Blood vessels,sex[T.male],1.106106,0.373477,3.059992e-03,9.179977e-03
37,ACE2,Submucosal Gland,age,0.005288,0.002329,2.319545e-02,2.783454e-02
49,ACE2,Lymphatics,age,0.066280,0.026666,1.293628e-02,2.587257e-02


In [30]:
full_res_lvl2_glm.loc[(full_res_lvl2_glm['gene'] == 'TMPRSS2') & (full_res_lvl2_glm['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
2,TMPRSS2,Myeloid,sex[T.male],0.551324,0.040177,7.472898e-43,2.241870e-42
8,TMPRSS2,Airway epithelium,sex[T.male],0.186310,0.011434,1.076613e-59,6.459678e-59
14,TMPRSS2,Alveolar epithelium,sex[T.male],0.095246,0.006786,9.499591e-45,1.424939e-44
15,TMPRSS2,Alveolar epithelium,age,0.021853,0.000217,0.000000e+00,0.000000e+00
20,TMPRSS2,Lymphoid,sex[T.male],0.331965,0.072183,4.246695e-06,8.493389e-06
21,TMPRSS2,Lymphoid,age,-0.005444,0.002462,2.701543e-02,3.241852e-02
26,TMPRSS2,Fibroblast lineage,sex[T.male],0.298200,0.054736,5.095326e-08,1.019065e-07
27,TMPRSS2,Fibroblast lineage,age,0.007127,0.001882,1.522644e-04,2.283966e-04
38,TMPRSS2,Submucosal Gland,sex[T.male],0.310393,0.023955,2.141115e-38,1.284669e-37
39,TMPRSS2,Submucosal Gland,age,0.001744,0.000579,2.580443e-03,3.870664e-03


# Level 3 annotation

In [31]:
cluster_key = 'ann_level_3'
clust_tbl = adata.obs[cluster_key].value_counts()
clusters = clust_tbl.index[clust_tbl > 1000]
ct_to_rm = clusters[[ct.startswith('1') or ct.startswith('2') for ct in clusters]]
clusters = clusters.drop(ct_to_rm.tolist()).tolist()
clusters

['Macrophages',
 'AT2',
 'Basal',
 'T cell lineage',
 'Monocytes',
 'Multiciliated lineage',
 'AT1',
 'Submucosal Secretory',
 'Innate lymphoid cells',
 'Secretory',
 'Capillary',
 'Mast cells',
 'B cell lineage',
 'Fibroblasts',
 'Dendritic cells',
 'Venous',
 'Lymphatic EC',
 'Arterial',
 'Rare',
 'Myofibroblasts',
 'MDC',
 'Airway smooth muscle']

In [32]:
adata_sub = adata[adata.obs.ann_level_3.isin(clusters),:]

adata_sub
adata_sub.obs.donor.nunique()
adata_sub.obs['sample'].nunique()

View of AnnData object with n_obs × n_vars = 895743 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'total_counts_scaled'

185

309

## Poisson GLM

In [33]:
# Poisson GLM loop
de_results_lvl3_glm = dict()

# Test over clusters
for clust in clusters:
    adata_tmp = adata_sub[adata_sub.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=10)
    if adata_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_tmp.obs.sex.value_counts())==1:
        print(f'{clust} only has 1 type of male/female sample.')
        continue        
        
    print(f'Testing {adata_tmp.n_vars} genes...')
    print(f'Testing in {adata_tmp.n_obs} cells...')    
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_tmp, #[idx_train],
        formula="~" + formula,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_tmp.X[:, i].todense(), #[idx_train, :], 
            exog=dmat[0],
            offset=np.log(adata_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()


        # Test over coefs
        for coef in tested_coef:
            de_results_temp = pois_results.wald_test(
                [x for i, x in enumerate(pois_model.exog_names) if dmat[1][i] in [coef]]
            )

            # Output the results nicely
            de_results_temp = pd.DataFrame({
                "gene": gene,
                "cell_identity": clust,
                "covariate": coef,
                "coef": pois_results.params[[y == coef for y in dmat[1]]],
                "coef_sd": pois_results.bse[[y == coef for y in dmat[1]]],                 
                "pval": de_results_temp.pvalue
            }, index= [clust+"_"+gene+"_"+coef])

            de_results_list.append(de_results_temp)

    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl3_glm[clust] = de_results
    
# Join the dataframes:
full_res_lvl3_glm = pd.concat([de_results_lvl3_glm[i] for i in de_results_lvl3_glm.keys()], ignore_index=True)

In cluster Macrophages:
male      113983
female     74988
Name: sex, dtype: int64
Testing 3 genes...
Testing in 188971 cells...

In cluster AT2:
female    112907
male       69217
Name: sex, dtype: int64
Testing 3 genes...
Testing in 182124 cells...

In cluster Basal:
female    91574
male      64804
Name: sex, dtype: int64
Testing 3 genes...
Testing in 156378 cells...

In cluster T cell lineage:
male      30198
female    29643
Name: sex, dtype: int64
Testing 3 genes...
Testing in 59841 cells...

In cluster Monocytes:
male      21798
female    21695
Name: sex, dtype: int64
Testing 3 genes...
Testing in 43493 cells...

In cluster Multiciliated lineage:
female    22405
male      19594
Name: sex, dtype: int64
Testing 3 genes...
Testing in 41999 cells...

In cluster AT1:
female    22611
male      17432
Name: sex, dtype: int64
Testing 3 genes...
Testing in 40043 cells...

In cluster Submucosal Secretory:
female    19901
male      13760
Name: sex, dtype: int64
Testing 3 genes...
Testing in 336

## Inspect some results

In [34]:
de_results_lvl3_glm.keys()

dict_keys(['Macrophages', 'AT2', 'Basal', 'T cell lineage', 'Monocytes', 'Multiciliated lineage', 'AT1', 'Submucosal Secretory', 'Innate lymphoid cells', 'Secretory', 'Capillary', 'Mast cells', 'B cell lineage', 'Fibroblasts', 'Dendritic cells', 'Venous', 'Lymphatic EC', 'Arterial', 'Rare', 'Myofibroblasts', 'MDC', 'Airway smooth muscle'])

In [35]:
full_res_lvl3_glm

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],0.264930,0.203802,1.936233e-01,2.499217e-01
1,ACE2,Macrophages,age,0.008422,0.006693,2.082681e-01,2.499217e-01
2,TMPRSS2,Macrophages,sex[T.male],-0.094569,0.057101,9.768750e-02,1.953750e-01
3,TMPRSS2,Macrophages,age,-0.001177,0.001983,5.528093e-01,5.528093e-01
4,CTSL,Macrophages,sex[T.male],-0.144491,0.002286,0.000000e+00,0.000000e+00
5,CTSL,Macrophages,age,0.003185,0.000079,0.000000e+00,0.000000e+00
6,ACE2,AT2,sex[T.male],0.868916,0.053569,3.608365e-59,1.082509e-58
7,ACE2,AT2,age,0.012644,0.001570,8.081616e-16,1.212242e-15
8,TMPRSS2,AT2,sex[T.male],0.022611,0.007279,1.895896e-03,1.895896e-03
9,TMPRSS2,AT2,age,0.022656,0.000234,0.000000e+00,0.000000e+00


In [36]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'ACE2',]

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],0.264930,0.203802,1.936233e-01,2.499217e-01
1,ACE2,Macrophages,age,0.008422,0.006693,2.082681e-01,2.499217e-01
6,ACE2,AT2,sex[T.male],0.868916,0.053569,3.608365e-59,1.082509e-58
7,ACE2,AT2,age,0.012644,0.001570,8.081616e-16,1.212242e-15
12,ACE2,Basal,sex[T.male],0.027527,0.047107,5.589858e-01,5.589858e-01
13,ACE2,Basal,age,0.004164,0.001286,1.209109e-03,1.450930e-03
18,ACE2,T cell lineage,sex[T.male],0.353216,0.528388,5.038283e-01,6.045940e-01
19,ACE2,T cell lineage,age,-0.006742,0.018993,7.226171e-01,7.226171e-01
24,ACE2,Monocytes,sex[T.male],0.834436,0.339375,1.394247e-02,2.091371e-02
25,ACE2,Monocytes,age,-0.023080,0.015470,1.357069e-01,1.357069e-01


In [37]:
full_res_lvl3_glm.loc[(full_res_lvl3_glm['gene'] == 'ACE2') & (full_res_lvl3_glm['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
6,ACE2,AT2,sex[T.male],0.868916,0.053569,3.608365e-59,1.082509e-58
7,ACE2,AT2,age,0.012644,0.001570,8.081616e-16,1.212242e-15
13,ACE2,Basal,age,0.004164,0.001286,1.209109e-03,1.450930e-03
24,ACE2,Monocytes,sex[T.male],0.834436,0.339375,1.394247e-02,2.091371e-02
30,ACE2,Multiciliated lineage,sex[T.male],0.230449,0.079931,3.937810e-03,6.342475e-03
31,ACE2,Multiciliated lineage,age,0.007502,0.002622,4.228317e-03,6.342475e-03
36,ACE2,AT1,sex[T.male],0.494016,0.182827,6.890467e-03,8.268561e-03
37,ACE2,AT1,age,0.014541,0.005878,1.336652e-02,1.336652e-02
43,ACE2,Submucosal Secretory,age,0.005288,0.002329,2.319545e-02,2.783454e-02
52,ACE2,Secretory,sex[T.male],0.241538,0.079763,2.460042e-03,3.690062e-03


In [38]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'TMPRSS2',]

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
2,TMPRSS2,Macrophages,sex[T.male],-0.094569,0.057101,9.768750e-02,1.953750e-01
3,TMPRSS2,Macrophages,age,-0.001177,0.001983,5.528093e-01,5.528093e-01
8,TMPRSS2,AT2,sex[T.male],0.022611,0.007279,1.895896e-03,1.895896e-03
9,TMPRSS2,AT2,age,0.022656,0.000234,0.000000e+00,0.000000e+00
14,TMPRSS2,Basal,sex[T.male],0.318730,0.020012,4.100876e-57,8.201751e-57
15,TMPRSS2,Basal,age,0.004448,0.000491,1.228279e-19,1.842418e-19
20,TMPRSS2,T cell lineage,sex[T.male],0.424947,0.086699,9.515144e-07,1.903029e-06
21,TMPRSS2,T cell lineage,age,-0.003390,0.003400,3.187745e-01,4.781618e-01
26,TMPRSS2,Monocytes,sex[T.male],1.130462,0.057860,5.215319e-85,3.129191e-84
27,TMPRSS2,Monocytes,age,0.005357,0.002650,4.319063e-02,5.182876e-02


In [39]:
full_res_lvl3_glm.loc[(full_res_lvl3_glm['gene'] == 'TMPRSS2') & (full_res_lvl3_glm['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
8,TMPRSS2,AT2,sex[T.male],0.022611,0.007279,1.895896e-03,1.895896e-03
9,TMPRSS2,AT2,age,0.022656,0.000234,0.000000e+00,0.000000e+00
14,TMPRSS2,Basal,sex[T.male],0.318730,0.020012,4.100876e-57,8.201751e-57
15,TMPRSS2,Basal,age,0.004448,0.000491,1.228279e-19,1.842418e-19
20,TMPRSS2,T cell lineage,sex[T.male],0.424947,0.086699,9.515144e-07,1.903029e-06
26,TMPRSS2,Monocytes,sex[T.male],1.130462,0.057860,5.215319e-85,3.129191e-84
32,TMPRSS2,Multiciliated lineage,sex[T.male],0.181666,0.018777,3.856170e-22,2.313702e-21
38,TMPRSS2,AT1,sex[T.male],0.629844,0.018878,4.451505e-244,2.670903e-243
39,TMPRSS2,AT1,age,0.011628,0.000694,4.967854e-63,1.490356e-62
44,TMPRSS2,Submucosal Secretory,sex[T.male],0.310393,0.023955,2.141115e-38,1.284669e-37


In [40]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'CTSL',]

,gene,cell_identity,covariate,coef,coef_sd,pval,adj_pvals
4,CTSL,Macrophages,sex[T.male],-0.144491,0.002286,0.000000e+00,0.000000e+00
5,CTSL,Macrophages,age,0.003185,0.000079,0.000000e+00,0.000000e+00
10,CTSL,AT2,sex[T.male],0.092925,0.013407,4.168612e-12,5.002334e-12
11,CTSL,AT2,age,-0.003468,0.000349,3.287226e-23,6.574452e-23
16,CTSL,Basal,sex[T.male],-0.178596,0.010448,1.649625e-65,4.948876e-65
17,CTSL,Basal,age,0.005529,0.000295,1.754298e-78,1.052579e-77
22,CTSL,T cell lineage,sex[T.male],0.382625,0.034855,4.896204e-28,1.468861e-27
23,CTSL,T cell lineage,age,0.030963,0.001406,1.578595e-107,9.471572e-107
28,CTSL,Monocytes,sex[T.male],-0.174573,0.010417,4.944319e-63,1.483296e-62
29,CTSL,Monocytes,age,-0.002606,0.000411,2.215098e-10,4.430196e-10


# Store results

In [47]:
full_res_lvl2_glm.to_csv(folder+'/'+output_folder+de_output_base+'_lvl2_full.csv')

In [48]:
full_res_lvl3_glm.to_csv(folder+'/'+output_folder+de_output_base+'_lvl3_full.csv')